# basic CNN for MNIST (17.01.10)

## Prepare Dataset
먼저 필요한 모듈들을 부르고, 그 데이터들을 Train Set과 Test Set을 나누었다. 

X는 데이터의 Image 들이 될 것이고, Y는 그 이미지의 Target(정답)이 될 것이다. 

예로 숫자 7이라는 X의 Y 값은 7가 될 것이다. 

하지만 one_hot 라는 옵션은 7 = [0, 0, 0, 0, 0, 0, 1, 0, 0, 0] 이런 식으로 만들어 주는 것이다. 

In [1]:
import tensorflow as tf
import numpy
import tflearn.datasets.mnist as mnist
trainX, trainY, testX, testY = mnist.load_data(one_hot=True)

데이터 구성을 한번 보았다. 

trainX를 보면 이미지 55000개의 이미지가 있고, 784 픽셀의 이미지라고 되어있는 것을 볼 수 있다. 

일반적으로 우리가 알고 있는 28 x 28 식의 2차원적인 이미지가 아닌, 한줄로 784개로 나열 되어있는 좀 특이 케이스의 이미지 데이터 셋이다. 

trainY에서 보면 10으로 되어있는 건 10개의 클래스라는 것을 알 수 있다. ( 0 ~ 9 까지 )

test set은 10000개 이다. 

In [3]:
print("trainX : \t", trainX.shape)
print("trainY : \t", trainY.shape)
print("testX : \t", testX.shape)
print("testY : \t", testY.shape)

trainX : 	 (55000, 784)
trainY : 	 (55000, 10)
testX : 	 (10000, 784)
testY : 	 (10000, 10)


## Modeling 

모델을 만들기 전에 먼저 weight와 bias를 만들어 줬다. 

In [4]:
weights = {
    # stddev 는 Standard Deviation의 약자로 랜덤한 숫자의 표준 편차를 정해준다. 
    # 랜덤한 Weight와 Bias를 Variable에 넣어준다. 우리가 학습해서 구하고 싶은 값들이기 때문이다. 
    'wc1': tf.Variable(tf.random_normal([3,3,1,64], stddev=0.1)),
    'wc2': tf.Variable(tf.random_normal([3,3,64,128], stddev=0.1)),
    'wd1': tf.Variable(tf.random_normal([7*7*128, 1024], stddev=0.1)),
    'wd2': tf.Variable(tf.random_normal([1024, 10], stddev=0.1))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([64], stddev=0.1)),
    'bc2': tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bd1': tf.Variable(tf.random_normal([1024], stddev=0.1)),
    'bd2': tf.Variable(tf.random_normal([10], stddev=0.1))
}

아래는 본격적인 모델링 설계다

In [5]:
def conv_basic(_input, _w, _b, _keepratio):
    # Input
    # 784로 들어오는 이미지 사이즈를 28 * 28 * 1 로 바꿔준다. 
    _input_r = tf.reshape(_input, [-1, 28, 28, 1])
    
    # Conv 1
    _conv1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_input_r, _w['wc1'], strides=[1,1,1,1], padding='SAME'), _b['bc1']))
    _pool1 = tf.nn.max_pool(_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    _drop1 = tf.nn.dropout(_pool1, _keepratio)
    
    # Conv 2
    _conv2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_drop1, _w['wc2'], strides=[1,1,1,1], padding='SAME'), _b['bc2']))
    _pool2 = tf.nn.max_pool(_conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    _drop2 = tf.nn.dropout(_pool2, _keepratio)
    
    # FC 1
    _dense = tf.reshape(_drop2, [-1, _w['wd1'].get_shape().as_list()[0]])
    _fc1 = tf.nn.relu(tf.nn.bias_add(tf.matmul(_dense, _w['wd1']), _b['bd1']))
    _fc1_drop = tf.nn.dropout(_fc1, _keepratio)
    
    # FC 2
    out = tf.add(tf.matmul(_fc1_drop, _w['wd2']), _b['bd2'])
    
    return out

모델링을 하고서, 학습방법이나 부가적인 Hyperparameter들을 조절 해주었다. 

In [6]:
learning_rate = 0.001  # Learning Rate을 정해준다. 
train_epochs = 5       # 총 몇 번 돌릴지 Epoch 수를 정해준다. 
batch_size = 100       # 몇 개 씩 쪼개어 학습할 지 Batch의 크기를 정해준다. 
display_step= 1        # 몇 번의 epoch 마다 결과를 보여줄 지 정해준다. 

x = tf.placeholder(tf.float32, [None, 784])  # Input을 담아줄 그릇(Placeholder)를 만들어준다. 
y = tf.placeholder(tf.float32, [None, 10])   # Ouput으로 꺼낼 때 필요한 그릇
keepratio = tf.placeholder(tf.float32)       # Dropout에 넣을 그릇을 또 만들어 준다. 

In [7]:
pred = conv_basic(x, weights, biases, keepratio)                            # 학습하고서 예측한 결과( y' ) 가 여기서 나온다. 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))     # Loss를 구해준다. 얼마나 틀렸는지를 Cross Entropy로 구했다.
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)   # 구한 Loss를 Optimizer로 학습 시켜준다. 
corr = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))                          # 예측한 결과( y' )가 정답인 y하고 맞는지 확인한다.  
accr = tf.reduce_mean(tf.cast(corr, tf.float32))                            # 맞는 갯수의 평균으로 정확도를 계산한다. 
init = tf.global_variables_initializer()                                    # 모든 Variables들을 초기화하는 것을 만든다. 

sess = tf.Session()
sess.run(init) # 학습을 시작하기 전에 Variable 들을 초기화 한다. 

Summary Ready


## Training Model

In [8]:
print("Start!")
for epoch in range(train_epochs):
    avg_cost = 0.
    total_batch = int(trainX.shape[0]/batch_size)
    training_batch = zip(range(0, len(trainX), batch_size), range(batch_size, len(trainX), batch_size))

    for start, end in training_batch:
        # 진짜 학습 시작!! 
        summary, _ = sess.run([merged, optm], feed_dict={x: trainX[start:end], y: trainY[start:end], keepratio:0.7})

        # Cost 평균값 업데이트
        avg_cost += sess.run(cost, feed_dict={x: trainX[start:end], y: trainY[start:end], keepratio:1.})/total_batch
        # Add Summary
        # 학습 과정 기록: ( 1(epoch) * 550(1개의 epoch 안에 batch 수) + 1(1번째 epoch) )
        summary_writer.add_summary(summary, epoch*total_batch+int(end/batch_size)) 
            
    # Display logs per epoch step
    if epoch % display_step == 0:
        print("Epoch: %03d/%03d, Cost: %.9f" % (epoch+1, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: trainX[start:end], y: trainY[start:end], keepratio:1.})
        print(" Training Accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accr, feed_dict={x: testX, y: testY, keepratio:1.})
        print(" Test Accuracy: %.3f" % (test_acc))

print("Optimization Finished")

Start!
Epoch: 000/005, Cost: 0.320982052
 Training Accuracy: 1.000
 Test Accuracy: 0.981
Epoch: 001/005, Cost: 0.050338285
 Training Accuracy: 1.000
 Test Accuracy: 0.987
Epoch: 002/005, Cost: 0.031639963
 Training Accuracy: 1.000
 Test Accuracy: 0.989
Epoch: 003/005, Cost: 0.023121420
 Training Accuracy: 1.000
 Test Accuracy: 0.991
Epoch: 004/005, Cost: 0.017677914
 Training Accuracy: 1.000
 Test Accuracy: 0.990
Optimization Finished
